In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [40]:
os.chdir("C:\\Users\\xzou\\Downloads\\dataSets\\training")

In [41]:
all_data = pd.read_csv("volume(table 6)_training.csv")

In [42]:
def time_wrapper(num):
    if num < 10:
        return "0" + str(num)
    else:
        return str(num)

In [43]:
def get_time_window(datetime):
    ymd = datetime.isoformat(" ").split(" ")[0]
    hour = time_wrapper(datetime.hour)
    minute = time_wrapper(datetime.minute)
    return ymd + " " + ":".join([hour, minute])

In [44]:
start_datetime = datetime(2016,9,19,0,0,0)
time_granularity = 20
tid_direction_dict = {}
for tid in [1,2,3]:
    for direction in [0,1]:
        tid_direction_data = all_data[(all_data.tollgate_id == tid) & (all_data.direction == direction)] \
                            .drop(["tollgate_id", "direction"], axis = 1)
        if tid_direction_data.shape[0] > 0:
            tid_direction_dict[(tid, direction)] = tid_direction_data.sort(["time"])
for key, tid_direction_df in tid_direction_dict.iteritems():
    time_list = tid_direction_df["time"].tolist()
    last_pos = 0
    pos = 0
    time_window = []
    missing_timewindow = []
    missing_records_dict = {}
    current_datetime = start_datetime
    while pos < tid_direction_df.shape[0]:
        this_time_interval_end = current_datetime + timedelta(minutes = time_granularity)
        this_datetime_str = time_list[pos]
        this_ymd = this_datetime_str.split(" ")[0]
        this_hms = this_datetime_str.split(" ")[1]
        int_this_ymd = map(int, this_ymd.split("-"))
        int_this_hms = map(int, this_hms.split(":"))
        this_datetime = datetime(int_this_ymd[0], int_this_ymd[1], int_this_ymd[2], 
                                 int_this_hms[0], int_this_hms[1], int_this_hms[2])
        if this_datetime >= current_datetime and this_datetime < this_time_interval_end:
            time_window.append(get_time_window(current_datetime))
            pos += 1
        else:
            if last_pos == pos: # insert a blank record representing the missing time window
                missing_timewindow.append(get_time_window(current_datetime))
            else:
                last_pos = pos
            current_datetime = this_time_interval_end

    tid_direction_df["time_window"] = time_window
    tid_direction_df["volume"] = [1] * len(time_window)
    tid_direction_v_dummies = pd.get_dummies(tid_direction_df["vehicle_model"], prefix="m")
    tid_direction_v_sep = pd.concat([tid_direction_df, tid_direction_v_dummies], axis = 1)
    if key[1] == 0:
        tid_direction_v_sep.drop(["time", "vehicle_model", "vehicle_type"], axis=1, inplace=True)
    else:
        tid_direction_v_sep.drop(["time", "vehicle_model"], axis=1, inplace=True)
    data_grouped_by_timewindow = tid_direction_v_sep.groupby(["time_window"]).sum().reset_index()
    data_grouped_by_timewindow.rename(columns={"vehicle_type": "vehicle_type_1"}, inplace=True)
    missing_records_dict["time_window"] = missing_timewindow
    for column in data_grouped_by_timewindow.columns:
        if column != "time_window":
            missing_records_dict[column] = [0] * len(missing_timewindow)
    missing_df = pd.DataFrame(missing_records_dict)
    whole_tid_direction_data = data_grouped_by_timewindow.append(missing_df, ignore_index=True).sort(["time_window"])
    tid_direction_dict[key] = whole_tid_direction_data
    whole_tid_direction_data.to_csv("T" + str(key[0]) + str(key[1]) + "ti_" + str(time_granularity) + ".csv", index=False)

C:\Users\xzou\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\xzou\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:51: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [26]:
tid_direction_dict[(1,0)]

,time_window,has_etc,volume,m_0,m_1,m_2,m_3,m_4,m_5,m_6,m_7
0,2016-09-19 00:00,1,5,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2016-09-19 00:10,3,8,0.0,6.0,1.0,0.0,0.0,1.0,0.0,0.0
2,2016-09-19 00:20,1,3,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-09-19 00:30,0,3,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016-09-19 00:40,1,6,0.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0
5,2016-09-19 00:50,0,3,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
6,2016-09-19 01:00,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2016-09-19 01:10,1,9,0.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0
8,2016-09-19 01:20,2,8,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0
9,2016-09-19 01:30,1,6,1.0,3.0,0.0,1.0,0.0,1.0,0.0,0.0


In [25]:
missing_df

,has_etc,m_1,m_2,m_3,m_4,m_5,m_6,m_7,time_window,vehicle_type_1,volume
0,0,0,0,0,0,0,0,0,2016-09-24 01:30,0,0
1,0,0,0,0,0,0,0,0,2016-09-30 21:40,0,0
2,0,0,0,0,0,0,0,0,2016-10-01 01:00,0,0
3,0,0,0,0,0,0,0,0,2016-10-01 01:10,0,0
4,0,0,0,0,0,0,0,0,2016-10-01 01:40,0,0
5,0,0,0,0,0,0,0,0,2016-10-01 01:50,0,0
6,0,0,0,0,0,0,0,0,2016-10-01 02:00,0,0
7,0,0,0,0,0,0,0,0,2016-10-01 02:30,0,0
8,0,0,0,0,0,0,0,0,2016-10-01 02:40,0,0
9,0,0,0,0,0,0,0,0,2016-10-01 06:50,0,0


In [32]:
tid_direction_dict[(2,0)].sort(["time_window"]).to_csv("for_look.csv", index=False)

C:\Users\xzou\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [33]:
tid_direction_dict[(2,0)]

,has_etc,m_0,m_1,m_2,m_3,m_4,m_7,time_window,volume
0,0,0.0,1.0,1.0,0.0,0.0,0.0,2016-09-19 00:00,2
1,1,0.0,1.0,0.0,0.0,0.0,0.0,2016-09-19 01:00,1
2,1,0.0,1.0,0.0,0.0,0.0,0.0,2016-09-19 01:40,1
3,1,0.0,1.0,0.0,0.0,0.0,0.0,2016-09-19 01:50,1
4,1,0.0,4.0,3.0,0.0,0.0,0.0,2016-09-19 05:00,7
5,0,0.0,5.0,2.0,0.0,0.0,0.0,2016-09-19 05:10,7
6,0,0.0,4.0,2.0,0.0,0.0,0.0,2016-09-19 05:20,6
7,0,0.0,4.0,2.0,0.0,0.0,0.0,2016-09-19 05:30,6
8,2,0.0,3.0,4.0,0.0,0.0,0.0,2016-09-19 05:40,7
9,3,0.0,11.0,3.0,0.0,0.0,0.0,2016-09-19 05:50,14


In [5]:
a = datetime(2016,9,18,10,50)

In [6]:
a.isoformat(" ")

'2016-09-18 10:50:00'

In [22]:
a = pd.DataFrame({"A":[1,2], "B":[3,4]})

In [23]:
a.append(pd.DataFrame({"A":[5,6],"B":[7,8]}), ignore_index=True)

,A,B
0,1,3
1,2,4
2,5,7
3,6,8


In [37]:
for i in a.columns:
    print i

A
B


In [39]:
a.columns.tolist()

['A', 'B']

In [14]:
a.append([[5,6],[7,8]], ignore_index=True)

,0,1,A,B
0,NaN,NaN,1.0,3.0
1,NaN,NaN,2.0,4.0
2,5.0,6.0,NaN,NaN
3,7.0,8.0,NaN,NaN


In [35]:
a["C"]

KeyError: 'C'

In [ ]:
pd.DataFrame